# **Ensemble Trees Exercise**

_John Andrew Dixon_

---

**Setup**

In [146]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [147]:
# Remote URL to the data
url ="https://docs.google.com/spreadsheets/d/e/2PACX-1vSQc1CsJ25nPMJcuJD04csFCysrzuInd_IQ_drLza49m_3R4MllPcuhduu4GozMJun3MgUJkGl0cw-d/pub?output=csv"
# Load and verify data
df = pd.read_csv(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   NOX      506 non-null    float64
 2   RM       506 non-null    float64
 3   AGE      506 non-null    float64
 4   PTRATIO  506 non-null    float64
 5   LSTAT    506 non-null    float64
 6   PRICE    506 non-null    float64
dtypes: float64(7)
memory usage: 27.8 KB


In [148]:
# Function for evaluating regression R^2 scores
def metrics(regressor, training, testing):
    """Function for evaluating a model's R^2 scores"""
    predictions = {
        "training": regressor.predict(training["X"]),
        "testing": regressor.predict(testing["X"])
    }

    metrics_df = pd.DataFrame({
        "R2": [regressor.score(training["X"], training["y"]), regressor.score(testing["X"], testing["y"])],
        "MAE": [mean_absolute_error(training["y"], predictions["training"]), mean_absolute_error(testing["y"], predictions["testing"])],
        "MSE": [mean_squared_error(training["y"], predictions["training"]), mean_squared_error(testing["y"], predictions["testing"])],
        "RMSE": [np.sqrt(mean_squared_error(training["y"], predictions["training"])), np.sqrt(mean_squared_error(testing["y"], predictions["testing"]))]
    }, index=["Training", "Testing"])
    return metrics_df

---

## **Tasks**

### **Try a Decision Tree, Bagged Tree, and Random Forest.**

In [149]:
# Create feature matrix and target vector
X = df.drop(columns="PRICE")
y = df["PRICE"]

In [150]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
training_set = {"X": X_train, "y": y_train}
testing_set = {"X": X_test, "y": y_test}

In [151]:
# Bring default trees and forest regressors into the mix
dt_default = DecisionTreeRegressor(random_state=42)
bt_default = BaggingRegressor(random_state=42)
rf_default = RandomForestRegressor(random_state=42)

In [152]:
# Fit all of them to the training data
dt_default.fit(X_train, y_train)
bt_default.fit(X_train, y_train)
rf_default.fit(X_train, y_train)
print(end='')

In [164]:
print("DecisionTreeRegressor")
display(metrics(dt_default, training_set, testing_set))
print("BaggingRegressor")
display(metrics(bt_default, training_set, testing_set))
print("RandomForestRegressor")
metrics(rf_default, training_set, testing_set)

DecisionTreeRegressor


,R2,MAE,MSE,RMSE
Training,1.000000,0.000000,0.000000,0.000000
Testing,0.619323,3.140945,26.657717,5.163111


BaggingRegressor


,R2,MAE,MSE,RMSE
Training,0.960676,1.103325,3.487356,1.867446
Testing,0.820421,2.315512,12.575417,3.546183


RandomForestRegressor


,R2,MAE,MSE,RMSE
Training,0.977134,0.953546,2.027774,1.423999
Testing,0.833853,2.207858,11.634795,3.410981


### **Tune each model to optimize performance on the test set.**
- After using a loop to tune each model, remember to create the best version of the model using the best hyperparameter values for the model based on the metrics you generated in your loop. The metrics from this best version model are what you will compare to the metrics of the other best version models to determine the overall best model.

#### **Decision Tree Tuning**

In [181]:
# DataFrame to hold the depth tuning
tuning = pd.DataFrame({
    "Depth": [],
    "Train R-Squared": [],
    "Test R-Squared": []
})

# Loop through pontential parameters
for number in range(1, 51):
    temp = DecisionTreeRegressor(max_depth=number, random_state=42)
    temp.fit(X_train, y_train)
    metrics_df = metrics(temp, training_set, testing_set).T
    tuning.loc[len(tuning)] = [number, metrics_df.iloc[0][0], metrics_df.iloc[0][1]]

# Get the depth that has the best test R^2
tuning.sort_values("Test R-Squared", ascending=False).head(3)

,Depth,Train R-Squared,Test R-Squared
6,7.0,0.958517,0.846377
9,10.0,0.986796,0.846010
10,11.0,0.991100,0.829736


#### **Bagged Tree Tuning**

#### **Random Forest Tuning**

### **Evaluate your best model using multiple regression metrics.**

### **Explain in a text cell how your model will perform if deployed by referring to the metrics.  Ex. How close can your stakeholders expect its predictions to be to the true value?**